In [1]:
!git clone https://github.com/ultralytics/yolov5  # clone
!cd yolov5 && pip install -r requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 16517, done.
remote: Counting objects: 100% (115/115), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 16517 (delta 47), reused 50 (delta 16), pack-reused 16402
Receiving objects: 100% (16517/16517), 15.18 MiB | 449.00 KiB/s, done.
Resolving deltas: 100% (11307/11307), done.


In [2]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="MhOCiRylsh0FoMSzqhBP")
project = rf.workspace("muhie-kk0uz").project("rugby-balls-zcnjx")
version = project.version(1)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rugby-balls-1 in yolov5pytorch:: 100%|██████████| 444/444 [00:00<00:00, 3821.66it/s]


In [4]:
!python yolov5/detect.py --weights yolov5s.pt --img 640 --conf 0.25 --source datasets/coco/images

detect: weights=['yolov5s.pt'], source=datasets/coco/images, data=yolov5/data/coco128.yaml, imgsz=[640, 640], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5/runs/detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.9.12 torch-2.2.1 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
image 1/216 /Users/muhie/betterwork/datasets/coco/images/IMG_5558_jpeg.rf.ede8f0f0b577d4f58333188434c3aae3.jpg: 640x640 1 person, 513.1ms
image 2/216 /Users/muhie/betterwork/datasets/coco/images/IMG_5559_jpeg.rf.485760e7ee1fb9ddb0b9f927e8a18cc8.jpg: 640x640 (no detections), 283.8ms
ima

In [8]:
!python yolov5/train.py --img 640 --epochs 3 --data datasets/coco128.yaml --weights yolov5s.pt

train: weights=yolov5s.pt, cfg=, data=datasets/coco128.yaml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=3, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, evolve_population=yolov5/data/hyps, resume_evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest, ndjson_console=False, ndjson_file=False
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.9.12 torch-2.2.1 CPU

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1

In [75]:
# %pip install ultralytics
import os
import cv2
import torch
from PIL import Image

# Model
# Model
model_path = 'yolov5/runs/train/exp2/weights/best.pt'  # Replace with your model path
model = torch.hub.load(model=model_path,source='local',repo_or_dir='dir')

im2 = 'ball2.jpeg' # OpenCV image (BGR to RGB)

# Inference
results = model(im2)  # batch of images

FileNotFoundError: [Errno 2] No such file or directory: '/Users/muhie/betterwork/dir/hubconf.py'

In [3]:
!python yolov5/export.py --weights best.pt --include onnx


export: data=yolov5/data/coco128.yaml, weights=['best.pt'], imgsz=[640, 640], batch_size=1, device=cpu, half=False, inplace=False, keras=False, optimize=False, int8=False, per_tensor=False, dynamic=False, simplify=False, opset=17, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['onnx']
YOLOv5 🚀 v7.0-294-gdb125a20 Python-3.9.12 torch-2.2.1 CPU

Fusing layers... 
[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs

PyTorch: starting from best.pt with output shape (1, 25200, 6) (13.7 MB)

ONNX: starting export with onnx 1.15.0...
ONNX: export success ✅ 1.3s, saved as best.onnx (27.2 MB)

Export complete (2.3s)
Results saved to /Users/muhie/betterwork
Detect:          python detect.py --weights best.onnx 
Validate:        python val.py --weights best.onnx 
PyTorch Hub:     model = torch.hub.load('ultralytics/

In [7]:
%pip install supervision==0.8.0
import cv2
import argparse

from ultralytics import YOLO
import supervision as sv
import numpy as np


ZONE_POLYGON = np.array([
    [0, 0],
    [0.5, 0],
    [0.5, 1],
    [0, 1]
])


def parse_arguments() -> argparse.Namespace:
    parser = argparse.ArgumentParser(description="YOLOv8 live")
    parser.add_argument(
        "--webcam-resolution", 
        default=[1280, 720], 
        nargs=2, 
        type=int
    )
    args = parser.parse_args()
    return args


def main():
    args = parse_arguments()
    frame_width, frame_height = args.webcam_resolution

    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)

    model = YOLO("best.pt")

    box_annotator = sv.BoxAnnotator(
        thickness=2,
        text_thickness=2,
        text_scale=1
    )

    zone_polygon = (ZONE_POLYGON * np.array(args.webcam_resolution)).astype(int)
    zone = sv.PolygonZone(polygon=zone_polygon, frame_resolution_wh=tuple(args.webcam_resolution))
    zone_annotator = sv.PolygonZoneAnnotator(
        zone=zone, 
        color=sv.Color.red(),
        thickness=2,
        text_thickness=4,
        text_scale=2
    )

    while True:
        ret, frame = cap.read()

        result = model(frame, agnostic_nms=True)[0]
        detections = sv.Detections.from_yolov8(result)
        labels = [
            f"{model.model.names[class_id]} {confidence:0.2f}"
            for _, confidence, class_id, _
            in detections
        ]
        frame = box_annotator.annotate(
            scene=frame, 
            detections=detections, 
            labels=labels
        )

        zone.trigger(detections=detections)
        frame = zone_annotator.annotate(scene=frame)      
        
        cv2.imshow("yolov8", frame)

        if (cv2.waitKey(30) == 27):
            break


if __name__ == "__main__":
    main()


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: pip3.10 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


usage: ipykernel_launcher.py [-h]
                             [--webcam-resolution WEBCAM_RESOLUTION WEBCAM_RESOLUTION]
ipykernel_launcher.py: error: unrecognized arguments: --f=/Users/muhie/Library/Jupyter/runtime/kernel-v2-73693aczkqmN8IqDG.json


SystemExit: 2